- Daisy:
`skimage.feature.daisy(image, step=4, radius=15, rings=3, histograms=8, orientations=8, normalization='l1', sigmas=None, ring_radii=None, visualize=False)` \\
Daisy recorre la imagen de a "step" pixeles considerando cada paso un punto clave, en el cual considera un anillo el cual rodea de otros anillos. Lo rodea con "rings" anillos de radio "radius".


# Config

A fines de elaboración, configuramos qué queremos que se recalcule y qué queremos usar desde Drive.

In [ ]:
recalcular_descriptores_daisy = False
guardar_descriptores_daisy = False
recalcular_descriptores_sift = False
guardar_descriptores_sift = False

recalcular_pipeline = False
recalcular_test = True
recalcular_mejores = True

# Imports

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from skimage.io import imread
from skimage.feature import SIFT
from skimage.feature import daisy
from skimage.transform import resize


from sklearn.cluster import MiniBatchKMeans
from sklearn.cluster import DBSCAN
from sklearn.cluster import AgglomerativeClustering
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.metrics import precision_recall_fscore_support
from scipy.spatial.distance import cdist
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

import time
import glob
import os
from tqdm import tqdm
import warnings
import math
import random

import pickle

warnings.filterwarnings('ignore')

# Configuración para gráficos
plt.rcParams['figure.figsize'] = (12, 8)
sns.set_style("whitegrid")

print("✅ Librerías importadas correctamente")

### ***UTIL***

In [ ]:
def indicesDeCategoriaEnLista(etiquetas, categoria):
  """Devuelve los indices donde la etiqueta sea igual que la categoria"""
  return [j for j, etiq in enumerate(etiquetas) if etiq == categoria]

def indexadoIndirecto(idx, lista_indices, lista_principal):
  """Devuelve el resultado de indexar indirectamente una lista"""
  return lista_principal[lista_indices[idx]]

### ***1. Carga y Exploración del Dataset***

**Uso del Drive**

Al tratarse de un dataset con tantos elementos, recomendamos utilizar el dataset directamente desde drive en vez de cargarlo cada vez que quieran trabajar con el. Intenten que todos los participantes del grupo lo tengan en el misma direccion para hacer mas fluido su trabajo.

Data set: [Rock Paper Scissors](https://www.kaggle.com/datasets/drgfreeman/rockpaperscissors?select=scissors)

Para los archivos .pkl: ...

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
def pathDeArchivoPorCategoria(data_path, categoria):
  """
  Devuelve la direccion de todos las imagenes en la carpeta de la categoria

  Returns:
    archivos: una lista con cada una de las direcciones de archivos.
  """

  categoria_path = os.path.join(data_path, categoria)

  archivos = glob.glob(os.path.join(categoria_path, "*.jpg")) + \
             glob.glob(os.path.join(categoria_path, "*.jpeg")) + \
             glob.glob(os.path.join(categoria_path, "*.png"))

  return archivos

def cargarImagenPorArchivo(archivo):
  """
  Devuelve la imagen cargada en blanco y negro, habiendo sido reescalada si fuera necesario.

  Returns:
    img: La imgen cargada por Archivo.
  """

  img = imread(archivo, as_gray=True)

  # Redimensionar si es necesario (opcional)
  if img.shape[0] > 400 or img.shape[1] > 400:
    img = resize(img, (300, 300), anti_aliasing=True)

  return img

In [ ]:
def cargarDataSet(data_path, categorias):
  """
  Carga todas las imágenes del dataset organizadas por categoría

  Returns:
    imagenes: lista de imágenes en escala de grises
    etiquetas: lista de etiquetas correspondientes
    nombres_archivo: lista con nombres de archivos para referencia
  """

  imagenes = []
  etiquetas = []
  nombres_archivo = []

  print(f"Cargando dataset desde: {data_path}")

  # Cargamos las imagenes de cada Categoria
  for categoria in categorias:

    archivos = pathDeArchivoPorCategoria(data_path, categoria)
    archivos = archivos

    print(f"  {categoria}: {len(archivos)} imágenes")

    for archivo in tqdm(archivos, desc=f"Cargando {categoria}"):

      try:
        img = cargarImagenPorArchivo(archivo)
        imagenes.append(img)
        etiquetas.append(categoria)
        nombres_archivo.append(os.path.basename(archivo))

      except Exception as e:
        print(f"Error cargando {archivo}: {e}")

  return imagenes, etiquetas, nombres_archivo


***CARGA***

### ADAPTAR CATEGORIAS, COLORES y PATH

In [ ]:
# Definir categorías de especies
CATEGORIAS = ['paper', 'rock', 'scissors']
COLORES_CATEGORIAS = ['#9e0142', '#d53e4f', '#f46d43']

# Paths del dataset
DATA_PATH = "dataset/"
TRAIN_PATH = os.path.join(DATA_PATH, "train")
TEST_PATH = os.path.join(DATA_PATH, "test")

# Cargar datasets de entrenamiento y prueba
print("=" * 50)
print("CARGANDO DATASET DE ENTRENAMIENTO")
print("=" * 50)
imagenes_train, etiquetas_train, nombres_train = cargarDataSet(TRAIN_PATH, CATEGORIAS)

print("\n" + "=" * 50)
print("CARGANDO DATASET DE PRUEBA")
print("=" * 50)
imagenes_test, etiquetas_test, nombres_test = cargarDataSet(TEST_PATH, CATEGORIAS)

print(f"\n📊 RESUMEN DEL DATASET:")
print(f"Training: {len(imagenes_train)} imágenes")
print(f"Testing: {len(imagenes_test)} imágenes")
print(f"Total: {len(imagenes_train) + len(imagenes_test)} imágenes")

***VISUALIZACION***

In [ ]:
# Visualización de ejemplos del dataset
def mostrarEjemplosDataSet(imagenes, etiquetas, categorias, n_ejemplos=2):
  """Muestra ejemplos de cada categoría del dataset"""

  fig, axes = plt.subplots(len(categorias), n_ejemplos, figsize=(15, 12))
  fig.suptitle('Ejemplos del Dataset por Categoría', fontsize=16, fontweight='bold')

  for i, categoria in enumerate(categorias):
    # Encontrar índices de esta categoría
    indices_categoria = indicesDeCategoriaEnLista(etiquetas, categoria)
    random.shuffle(indices_categoria)

    # Mostramos imagenes de Ejemplos por categoria
    for j in range(n_ejemplos):

        img = indexadoIndirecto(j, indices_categoria, imagenes)
        axes[i, j].imshow(img, cmap='gray', clim=(0,1))
        axes[i, j].set_title(f'{categoria}', fontweight='bold')
        axes[i, j].axis('off')

  plt.tight_layout()
  plt.show()

In [ ]:
mostrarEjemplosDataSet(imagenes_test, etiquetas_test, CATEGORIAS)

### ***2. Extracción de Características Daisy***
Con las imagenes cargadas, empezaremos por extraer cada uno de sus keypoints y descriptores de Daisy correspondientes. Luego utilizaremos estos datos para crear nuestras palabras visuales.

In [ ]:
def extraerDaisyDataSet(imagenes, step = 16, image_index=0, **kwargs):
    keypoints = []
    descriptores = []
    stats = {cat: {'total_keypoints': 0, 'promedio_keypoints': 0} for cat in CATEGORIAS}

    for i, img in enumerate(imagenes):
      if i == image_index:
        desc, image_daisy = daisy(img, step=step, visualize=True, **kwargs)
        plt.imsave('daisy_visualizacion.png', image_daisy, cmap='gray')
      else:
        desc = daisy(img, step = step , **kwargs)

      desc_flat = desc.reshape(-1, desc.shape[-1]).astype(np.float32)
      descriptores.append(desc_flat)

      # Guardar los keypoints (coordenadas de la grilla)
      x, y = np.meshgrid(np.arange(0, img.shape[1], step), np.arange(0, img.shape[0], step))
      puntos = np.vstack([x.ravel(), y.ravel()]).T
      keypoints.append(puntos)
      # Estadísticas

      stats[CATEGORIAS[i % len(CATEGORIAS)]]['total_keypoints'] += len(desc_flat)
    # Calcular promedios
    for cat in CATEGORIAS:
        stats[cat]['promedio_keypoints'] = stats[cat]['total_keypoints'] / max(1, sum([1 for e in imagenes if cat in str(e)]))
    return keypoints, descriptores, stats

In [ ]:
if recalcular_descriptores_daisy:

  # Extraer Daisy del dataset de entrenamiento
  print("\n" + "=" * 50)
  print("EXTRACCIÓN DE CARACTERÍSTICAS Daisy - TRAINING")
  print("=" * 50)

  keypoints_train_daisy, descriptores_train_daisy, stats_train_daisy = extraerDaisyDataSet(imagenes_train, step=45, image_index = 5, radius=30, rings=2, histograms=6, orientations=8)

  if guardar_descriptores_daisy:
    # Guardamos así no lo calculamos todo el tiempo
    ruta = 'daisy_train.pkl'
    with open(ruta, 'wb') as f:
      pickle.dump((keypoints_train_daisy, descriptores_train_daisy, stats_train_daisy), f)

  # Mostrar estadísticas
  print("\n📊 ESTADÍSTICAS DE EXTRACCIÓN Daisy:")
  for categoria in CATEGORIAS:
    total = stats_train_daisy[categoria]['total_keypoints']
    promedio = stats_train_daisy[categoria]['promedio_keypoints'] # corregir promedio
    print(f"{categoria}: {total} keypoints total, {promedio:.1f} promedio por imagen")

In [ ]:
if not recalcular_descriptores_daisy:

   # Cargar los resultados

  ruta = 'daisy_train.pkl'

  with open(ruta, 'rb') as f:
    keypoints_train_daisy, descriptores_train_daisy, stats_train_daisy = pickle.load(f)

In [ ]:
def extraerSIFTImagen(imagen):
  """
  Extrae características SIFT de una imagen

  Returns:
    keypoints: coordenadas de los puntos clave
    descriptores: descriptores SIFT (128 dimensiones cada uno)
  """

  sift = SIFT()

  sift.detect_and_extract(imagen)

  return sift.keypoints, sift.descriptors

In [ ]:
def extraerSIFTDataSet(imagenes, etiquetas):
  """
  Extrae SIFT de todo el dataset

  Returns:
    keypoints_dataset: lista de keypoints por imagen
    descriptores_dataset: lista de descriptores por imagen
    estadisticas: diccionario con estadísticas de extracción
  """

  keypoints_dataset = []
  descriptores_dataset = []

  # Estadísticas por categoría
  stats = {cat: {'total_keypoints': 0, 'num_images': 0} for cat in CATEGORIAS}

  print("Extrayendo características SIFT...")

  # i: indice | img: imagen correspondiente | etiqueta: etiqueta correspondiente
  for i, (img, etiqueta) in enumerate(tqdm(zip(imagenes, etiquetas), total=len(imagenes))):

    keypoints, descriptores = extraerSIFTImagen(img)

    keypoints_dataset.append(keypoints)
    descriptores_dataset.append(descriptores)

    # Actualizar estadísticas
    stats[etiqueta]['total_keypoints'] += len(keypoints)
    stats[etiqueta]['num_images'] += 1

  # Calcular promedios
  for categoria in stats:
    if stats[categoria]['num_images'] > 0:
      stats[categoria]['promedio_keypoints'] = stats[categoria]['total_keypoints'] / stats[categoria]['num_images']

  return keypoints_dataset, descriptores_dataset, stats

In [ ]:
if recalcular_descriptores_sift:

  # Extraer Daisy del dataset de entrenamiento
  print("\n" + "=" * 50)
  print("EXTRACCIÓN DE CARACTERÍSTICAS Sift - TRAINING")
  print("=" * 50)

  keypoints_train_sift, descriptores_train_sift, stats_train_sift = extraerSIFTDataSet(imagenes_train, etiquetas_train)

  if guardar_descriptores_daisy:
    # Guardamos así no lo calculamos todo el tiempo
    ruta = 'sift_train.pkl'
    with open(ruta, 'wb') as f:
      pickle.dump((keypoints_train_sift, descriptores_train_sift, stats_train_sift), f)

  # Mostrar estadísticas
  print("\n📊 ESTADÍSTICAS DE EXTRACCIÓN Sift:")
  for categoria in CATEGORIAS:
    total = stats_train_sift[categoria]['total_keypoints']
    promedio = stats_train_sift[categoria]['promedio_keypoints'] # corregir promedio
    print(f"{categoria}: {total} keypoints total, {promedio:.1f} promedio por imagen")

In [ ]:
if not recalcular_descriptores_sift:

   # Cargar los resultados

   ruta = 'sift_train.pkl'

   with open(ruta, 'rb') as f:
     keypoints_train_sift, descriptores_train_sift, stats_train_sift = pickle.load(f)

***VISUALIZACION***

In [ ]:
# Visualizar keypoints en ejemplos
def mostrar_keypoints_ejemplos(imagenes, keypoints, etiquetas, categorias):
  """Muestra keypoints detectados en ejemplos de cada categoría"""

  fig, axes = plt.subplots(2, len(categorias), figsize=(20, 10))
  fig.suptitle('Keypoints Daisy detectados por categoría', fontsize=16, fontweight='bold')

  for i, categoria in enumerate(categorias):
    indices_categoria = indicesDeCategoriaEnLista(etiquetas, categoria)
    random.shuffle(indices_categoria)

    for row in range(2):
      img = indexadoIndirecto(row, indices_categoria, imagenes)
      kp = indexadoIndirecto(row, indices_categoria, keypoints)

      # Imagen original
      axes[row, i].imshow(img, cmap='gray')

      # Keypoints
      axes[row, i].scatter(kp[:, 0], kp[:, 1], s=10, c='red', alpha=0.7)

      axes[row, i].set_title(f'{categoria} ({len(kp)} keypoints)')
      axes[row, i].axis('off')

  plt.tight_layout()
  plt.show()

print("\n" + "=" * 50)
print("VISUALIZACIÓN DE KEYPOINTS")
print("=" * 50)
mostrar_keypoints_ejemplos(imagenes_train, keypoints_train_daisy, etiquetas_train, CATEGORIAS)

### ***3. Cross Validation***

Para poder encontrar los mejores parametros a usar en el categorizador sin caer en la trampa de **sobreajustarse a los datos de test**, utilizaremos la tecnica de **Cross Validation**.

Dividiremos nuestro set de entrenamiento en 5 subsets balanceados en categoria y aleatorios. Por cada uno de los subsets, tomaremos el resto de los 4 para **entrenar un categorizador** utilizando los meta-parametros a testear (numero de clusters, batch size, etc) y los testearemos sobre el subset apartado.

Por ultimo, utilizaremos la combinacion de meta-parametros que mejor accuracy hayan conseguido en **promedio**.

Con esta tecnica aumentamos las chances de generalizar sin tener que aumentar los datos de entrenamiento, ya que estariamos eligiendo los parametros que mayor capacidad de generalizacion obtuvieron.

In [ ]:
def getPorcionDeSubset(lista, indices_cat, cat_subset_inicio, cat_subset_fin):
  """
    Devuelve una sublista del arreglo original siendo indexada indirectamente
    por un arreglo con los indices de una categoria en especifico
  """

  return [indexadoIndirecto(idx, indices_cat, lista) for idx in range(cat_subset_inicio, cat_subset_fin)]

def crearSubsets(imagenes, etiquetas, nombres, keypoints, descriptores, n_subsets=5):
  """Devuelve una lista con los subsets creados al dividir el dataset de entrenamiento original"""

  # Encontramos los indices que le pertenecen a cada categoria (3
  # Encontramos cuantos item de cada categoria deberia haber por subset
  indices_categorias = {cat: indicesDeCategoriaEnLista(etiquetas,cat) for cat in CATEGORIAS}
  categoria_por_subset = {cat: math.ceil(len(indices_categorias[cat]) / n_subsets) for cat in CATEGORIAS}

  # Reordenamos los indices de las categorias de forma aleatoria
  for cat in CATEGORIAS:
    random.shuffle(indices_categorias[cat])

  subsets = []

  for subset_index in range(n_subsets):
    subset = {
        'imagenes': [],
        'etiquetas': [],
        'nombres': [],
        'keypoints': [],
        'descriptores': [],
        'stats': {cat: {'num_images': 0} for cat in CATEGORIAS}
      }

    for cat in CATEGORIAS:

      cat_indices = indices_categorias[cat]
      cat_subset_inicio = subset_index * categoria_por_subset[cat]
      cat_subset_fin = min(cat_subset_inicio + categoria_por_subset[cat], len(indices_categorias[cat]))

      cat_etiquetas = getPorcionDeSubset(etiquetas, cat_indices, cat_subset_inicio, cat_subset_fin)
      cat_descriptores = getPorcionDeSubset(descriptores, cat_indices, cat_subset_inicio, cat_subset_fin)
      cat_nombres = getPorcionDeSubset(nombres, cat_indices, cat_subset_inicio, cat_subset_fin)
      cat_imagenes = getPorcionDeSubset(imagenes, cat_indices, cat_subset_inicio, cat_subset_fin)
      cat_keypoints = getPorcionDeSubset(keypoints, cat_indices, cat_subset_inicio, cat_subset_fin)

      subset['etiquetas'] += (cat_etiquetas)
      subset['nombres'] += (cat_nombres)
      subset['descriptores']+=(cat_descriptores)
      subset['imagenes']+=(cat_imagenes)
      subset['keypoints']+=(cat_keypoints)

      subset['stats'][cat]['num_images'] = len(cat_imagenes)

    subsets.append(subset)

  return subsets

In [ ]:
subsets_daisy = crearSubsets(imagenes_train, etiquetas_train, nombres_train, keypoints_train_daisy, descriptores_train_daisy)
subsets_sift = crearSubsets(imagenes_train, etiquetas_train, nombres_train, keypoints_train_sift, descriptores_train_sift)

for i in range(len(subsets_daisy)):
    subset = subsets_daisy[i]

    print(f"Subset {i}")
    print(f"Imagenes Totales: {len(subset['etiquetas'])}")

    for categoria in CATEGORIAS:
      print(f"\t{categoria}: {subset['stats'][categoria]['num_images']} imagenes")

    print("\n")

In [ ]:
def imagenesDeEjemploSubSet(subset, n_ejemplos=4):
  """Muestra ejemplos de cada categoría del subset"""

  fig, axes = plt.subplots(len(CATEGORIAS), n_ejemplos, figsize=(15, 12))
  fig.suptitle('Ejemplos del Sub Dataset por Categoría', fontsize=16, fontweight='bold')

  for i, categoria in enumerate(CATEGORIAS):
    # Encontrar índices de esta categoría
    indices_categoria = indicesDeCategoriaEnLista(subset['etiquetas'], categoria)
    random.shuffle(indices_categoria)

    # Mostramos imagenes de Ejemplos por categoria
    for j in range(n_ejemplos):

        img = indexadoIndirecto(j, indices_categoria, subset['imagenes'])
        axes[i, j].imshow(img, cmap='gray', clim=(0,1))
        axes[i, j].set_title(f'{categoria}', fontweight='bold')
        axes[i, j].axis('off')

  plt.tight_layout()
  plt.show()

In [ ]:
imagenesDeEjemploSubSet(subsets_daisy[0])

In [ ]:
def unirSubsets(subsets):
  """Devuelve un subset conformado por la union de los subset en la lista"""

  subset_union = {
        'imagenes': [],
        'etiquetas': [],
        'nombres': [],
        'keypoints': [],
        'descriptores': [],
        'stats': {cat: {'num_images': 0} for cat in CATEGORIAS}
      }

  for subset in subsets:

    subset_union['etiquetas'] += subset['etiquetas']
    subset_union['nombres']+= subset['nombres']
    subset_union['descriptores']+= subset['descriptores']
    subset_union['imagenes']+= subset['imagenes']
    subset_union['keypoints']+= subset['keypoints']

    for cat in CATEGORIAS:
      subset_union['stats'][cat]['num_images'] += len(subset['etiquetas'])

  return subset_union


### ***4. Creación del Vocabulario Visual***

Ya teniendo los descriptores, podemos clasificarlos para comprender que keypoints eran similares a traves de las imagenes y empezar a hablar sobre el contenido de ellas.


---

***Construccion de Vocabulario Visual*** ➡️ *Representacion BOW* ➡️ *Procesamiento de la Representacion* ➡️ *Entrenamiento de Categorizador* ➡️ *Validacion*

In [ ]:
# Crear matriz global de descriptores para clustering
def crearMatrizDescriptores(descriptores_dataset):
    """
    Combina todos los descriptores en una matriz global para poder realizar clustering
    Donde cada fila es un descriptor
    """
    matriz_descriptores = []

    for descriptores_imagen in descriptores_dataset:
      matriz_descriptores.append(descriptores_imagen)

    matriz_descriptores = np.vstack(matriz_descriptores)

    return matriz_descriptores

In [ ]:
def construirVocabularioAgglomerativeClustering(descriptores, n_clusters=100, metric='euclidean', linkage='ward'):
    """
    Construye vocabulario visual usando Agglomerative Clustering.

    Returns:
        modelo: objeto con labels y centroides de los clusters
    """

    print(f"Construyendo vocabulario con Agglomerative Clustering (n_clusters={n_clusters})...")
    modelo = AgglomerativeClustering(n_clusters=n_clusters, linkage=linkage)
    labels = modelo.fit_predict(descriptores)


    centroides = np.zeros((n_clusters, descriptores.shape[1]), dtype=np.float32)
    for i in range(n_clusters):
        grupo = descriptores[labels == i]
        if len(grupo) > 0:
            centroides[i] = grupo.mean(axis=0).astype(np.float32)

    return {"modelo": modelo, "centroides": centroides}

In [ ]:
def construirVocabularioMiniBatchKMeans(descriptores, k=200, batch_size=1024, random_state=42):
  """
  Construye vocabulario visual usando K-means

  Args:
    descriptores: matriz de descriptores
    k: número de clusters (palabras visuales)
    batch_size: Cantidad de keypoints aleatorios que utilizara para encontrar los clusters (chico para hacer pruebas, luego incrementar)
    random_state: semilla para reproducibilidad

  Returns:
    kmeans: Categorizador K-Means entrenado para categorizar los descriptores a su centroido mas cercano
  """
  print(f"Construyendo vocabulario con k={k} palabras visuales...")

  kmeans = MiniBatchKMeans(
      n_clusters=k,
      random_state=random_state,
      batch_size=batch_size,
      max_iter=300,
      n_init=5,
      verbose=0
      )

  kmeans.fit(descriptores)

  return kmeans

### ***5. Representación Bag of Words***
Ya teniendo nuestro corpus o palabras visuales, podremos empezar a re-describir nuestras imagenes segun este. Gracias a esto podemos "hablar" de todas nuestras imagenes en el mismo "lenguaje", el nuevo descriptor.

Para la descripcion de una imagen utilizaremos la frecuencia de cada palabra en el vocabulario.

---

*Construccion de Vocabulario Visual* ➡️ ***Representacion BOW*** ➡️ *Procesamiento de la Representacion* ➡️ *Entrenamiento de Categorizador* ➡️ *Validacion*

### FALTA OBTENER n_clusters A PARTIR DEL MODELO ENTRENADO

In [ ]:
def descriptoresABOWAgglomerativeClustering(descriptores_imagen, vocabulario):
    """
    Convierte descriptores de una imagen a representación BoW

    Args:
        descriptores_imagen: np.array de descriptores Daisy de una imagen
        vocabulario: modelo Agglomerative Clustering entrenado (modelo del vocabulario visual)

    Returns:
        histograma: vector de frecuencias de palabras visuales (BoW)
    """
    centroides = vocabulario["centroides"].astype(np.float32)

    # Calcular distancias de cada descriptor al centroide más cercano
    distancias = cdist(descriptores_imagen, centroides, metric='euclidean').astype(np.float32) # (n_descriptores, n_clusters)

    # Asignar cada descriptor al centroide más cercano (palabra visual)
    asignaciones = np.argmin(distancias, axis=1)

    # Construir histograma de frecuencias de palabras visuales
    histograma, _ = np.histogram(asignaciones, bins=np.arange(len(centroides) + 1))

    return histograma.astype(np.float32)

In [ ]:
def descriptoresABOWMiniBatchKMeans(descriptores_imagen, vocabulario):
    """
    Convierte descriptores de una imagen a representación BoW

    Args:
        descriptores_imagen: np.array de descriptores SIFT de una imagen
        vocabulario: objeto MiniBatchKMeans entrenado (modelo del vocabulario visual)

    Returns:
        histograma: vector de frecuencias de palabras visuales (BoW)
    """
    n_clusters = vocabulario.n_clusters

    # Asignar cada descriptor al centroide más cercano (palabra visual)
    asignaciones = vocabulario.predict(descriptores_imagen) #COMPLETAR ...

    # Construir histograma de frecuencias de palabras visuales
    histograma, _ = np.histogram(asignaciones, bins=np.arange(n_clusters + 1)) #COMPLETAR ...

    return histograma.astype(float)

In [ ]:
def crearMatrizBOW(descriptores_dataset, vocabulario, clustering=None):
    """
    Crea matriz BoW para todo el dataset
    Donde Cada fila es la frecuencia del vocabulario en una imagen
    """
    matriz_bow = []

    print("Creando representaciones BoW...")

    for descriptores_imagen in tqdm(descriptores_dataset):
      if clustering == "agglomerative_clustering":
        bow = descriptoresABOWAgglomerativeClustering(descriptores_imagen, vocabulario)
      else:
        bow = descriptoresABOWMiniBatchKMeans(descriptores_imagen, vocabulario)
      matriz_bow.append(bow.astype(np.float32))
      del bow

    return np.array(matriz_bow)

### ***6. TF-IDF***
Con la Matriz de Representacion de Bag Of Words creada, podemos empezar a transformarla para intentar conseguir un mejor puntaje de forma barata. Una de nuestras opciones es aplicarle **TF-IDF**, presentado en la clase practica, con la intencion de darle mas importancia a las palabras visuales menos frecuentes y bajar el impacto de palabras comunes que no aportan informacion. Si pensamos en una analogia con texto, seria el equivalente de "bajarle el precio" a conectores, pronombres, articulos y etc.

$$
\text{Tf-idf}_{t,d} = \text{tf}_{t,d} ⋅ log(\frac{N}{\text{df}_t})
$$

donde
- $\text{Tf-idf}_{t,d}$ sera el nuevo valor de la palabra $t$ para la imagen $d$
- $\text{tf}_{t,d}$ era la frecuencia de la palabra $t$ para la imagen $d$
- $N$ es la cantidad de imagenes totales
- $\text{df}_t$ es la cantidad de imagenes en la que la palabra $t$ aparece al menos una vez.

---

*Construccion de Vocabulario Visual* ➡️ *Representacion BOW* ➡️ ***Procesamiento de la Representacion*** ➡️ *Entrenamiento de Categorizador* ➡️ *Validacion*

In [ ]:
def aplicar_tfidf(matriz_bow):
  """
  Aplica transformación TF-IDF a matriz de frecuencias

  Args:
    matriz_bow: matriz de frecuencias (n_images x n_words)

  Returns:
    matriz_tfidf: matriz con pesos TF-IDF
  """

  matriz_tfidf = matriz_bow.copy()

  n, m = matriz_bow.shape

  df = np.count_nonzero(matriz_bow>0, axis=0)

  df[df==0] = 1

  idf = np.log(n / df)

  matriz_tfidf = matriz_tfidf * idf


  return matriz_tfidf

### ***7. Entrenamiento de un Clasificador***

Ya habiendo aplicado todo el **post-procesamiento** que quisieramos a nuestra aplicacion, solo nos queda usarla para entrenar un **Categorizador de Escenas** que luego utilizaremos para hacer predicciones con imagenes entrantes nuevas.

En este caso usaremos una **SVC** por su flexibilidad y por ya haber sido presentada en la Tarea anterior. Recordemos que una **SVC** encuentra los mejores Hiper Planos para separar los puntos de cada categoria, dejandonos aplicarle una **transformacion a los datos de entradas a un espacio de dimensionalidad mayor** si nos es necesario para encontrar hiperplanos validos.

---

*Construccion de Vocabulario Visual* ➡️ *Representacion BOW* ➡️ *Procesamiento de la Representacion* ➡️ ***Entrenamiento de Categorizador*** ➡️ *Validacion*

In [ ]:
def entrenar_svm(descriptores_train, etiquetas_train):
    """
    Entrena un clasificador SVM

    Returns:
        clf: pipeline con SVC
    """

    # make_pipeline les permite crear un pipeline sobre el propio categorizador SVC
    # por si quieren hacer un procesamiento mas general de los datos antes de enviarlos a la SVC
    clf = make_pipeline(SVC(kernel='linear', probability=True))
    clf.fit(descriptores_train, etiquetas_train)
    return clf

### ***8. Validacion***

Con nuestro Clasificador entrenado, solo queda probarlo con los datos de validacion. Para esto tendremos que representar nuestras imagenes de validacion utilizando el vocabulario y las transformaciones de post-procesamiento con las que se entreno la SVM.

Luego podremos analizar con mas detalles que puntos debiles tiene nuestro clasificador, por ejemplo, cuales son las categorias que mas se confunde entre si.

---

*Construccion de Vocabulario Visual* ➡️ *Representacion BOW* ➡️ *Procesamiento de la Representacion* ➡️ *Entrenamiento de Categorizador* ➡️ ***Validacion***

In [ ]:
def evaluar_clasificador_svm(clasificador, descriptores_bow_test, etiquetas_test, nombre_experimento=""):
  """
  Evalúa el clasificador SVM
  Args:
    clasificador: Clasificador a Evaluar
    descriptores_bow_test: Descriptores de BoW a categorizar
    etiquetas_test: Ground Truth para cada descriptor
    nombre_experimento: Nombre opcional a imprimr

  Returns:
    predicciones: Etiquetas predichas por el clasificador para cada muestra del conjunto de prueba.
    confianzas: Probabilidad más alta asignada por el modelo a la clase predicha para cada muestra (valor entre 0 y 1).
    accuracy: Proporción de etiquetas correctamente clasificadas sobre el total de muestras: (n° aciertos) / (n° total).
    precision: Para cada categoría, proporción de verdaderos positivos entre todos los predichos como esa categoría: TP / (TP + FP).
    recall: Para cada categoría, proporción de verdaderos positivos entre los casos reales de esa categoría: TP / (TP + FN).
    f1: Para cada categoría, media armónica entre precisión y recall: 2 * (precision * recall) / (precision + recall).
    support: Número real de muestras en cada categoría dentro del conjunto de prueba (distribución real por clase).
    confusion_matrix: Matriz de Confusion
  """

  print(f"Evaluando clasificador: {nombre_experimento}")

  # Predicción
  predicciones = clasificador.predict(descriptores_bow_test)

  # Confianzas: tomamos la probabilidad de la clase predicha
  confianzas_prob = clasificador.predict_proba(descriptores_bow_test)
  confianzas = confianzas_prob.max(axis=1)

  # Métricas
  accuracy = accuracy_score(etiquetas_test, predicciones)

  precision, recall, f1, support = precision_recall_fscore_support(
      etiquetas_test, predicciones, average=None, labels=CATEGORIAS
  )

  # Creamos una Matriz de Confusion para ver en que categorias se confunde nuestro clasificador
  cm = confusion_matrix(etiquetas_test, predicciones, labels=CATEGORIAS)

  return {
    'predicciones': predicciones,
    'confianzas': confianzas,
    'accuracy': accuracy,
    'precision': precision,
    'recall': recall,
    'f1': f1,
    'support': support,
    'confusion_matrix': cm
  }

### ***8 Bis. Analisis de Resultados***
Ya teniendo una muestra de como funciona nuestro categorizador, analizemos en mas detalle su desempeño.

#### **Funciones de Analisis**

In [ ]:
def crearTablaResultados(resultados, ks=None, metodos=None):
    """
    Crea tabla resumen con todos los resultados
    resultados: dict {metodo: métricas}
    ks: dict opcional {metodo: valor_k}, si no se pasa, muestra '-'
    metodos: lista opcional [(clave, titulo_legible)]
    """
    if ks is None:
        ks = {}

    if metodos is None:
        metodos = [(m, m) for m in resultados.keys()]

    print("\n📊 TABLA RESUMEN DE RESULTADOS")
    print("=" * 90)
    print(f"{'Config':<35} {'Accuracy':<10} {'Precisión':<12} {'Recall':<10} {'F1-Score':<10}")

    for (metodo, titulo) in metodos:
        res = resultados[metodo]
        k_val = ks.get(metodo, '-')
        config = f"{titulo} (k={k_val})"

        acc = res['accuracy']
        prec = np.mean(res['precision'])
        rec = np.mean(res['recall'])
        f1 = np.mean(res['f1'])

        print(f"{config:<35} {acc:<10.3f} {prec:<12.3f} {rec:<10.3f} {f1:<10.3f}")

    print("-" * 90)
    print()

In [ ]:
def plotConfusionMatrices(resultados, k_valor, metodos):
    """Plotea matrices de confusión para un valor de k específico"""

    fig, axes = plt.subplots(1, len(metodos), figsize=(15, 6))
    if len(metodos) == 1:
        axes = [axes]
    for i, (metodo, titulo) in enumerate(metodos):

        cm = resultados[metodo]['confusion_matrix']
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                   xticklabels=[cat for cat in CATEGORIAS],
                   yticklabels=[cat for cat in CATEGORIAS],
                   ax=axes[i])

        axes[i].set_title(f'{titulo} (k={k_valor})')
        axes[i].set_xlabel('Predicción')
        axes[i].set_ylabel('Verdadero')

    plt.tight_layout()
    plt.show()

In [ ]:
def analizar_errores(resultados, metodo, testing_set):
  """Devuelve los errores de clasificación"""

  resultado = resultados[metodo]
  predicciones = resultado['predicciones']

  etiquetas = testing_set['etiquetas']
  imagenes = testing_set['imagenes']
  nombres = testing_set['nombres']

  # Encontrar errores
  errores = []
  for i, (real, pred) in enumerate(zip(etiquetas, predicciones)):
    if real != pred:
      errores.append({
        'real': real,
        'predicho': pred,
        'imagen': imagenes[i],
        'imagen_nombre': nombres[i],
        'confianza': resultado['confianzas'][i]
      })

  return errores

In [ ]:
def imprimirErrores(errores, k, metodo, total, name=""):
  """Imprime informacion sobre los errores"""
  if not (k == None):
    print(f"\n🔍 ANÁLISIS DE ERRORES ({metodo.upper()}, {name.upper()}, k={k})")
  else:
    print(f"\n🔍 ANÁLISIS DE ERRORES ({metodo.upper()}, {name.upper()})")

  print("-" * 50)
  print(f"Total de errores: {len(errores)} de {total} imágenes")
  print(f"Accuracy: {(total - len(errores))/total:.3f}")



  # Análisis de confusiones más comunes
  confusiones = {}
  for error in errores:
    par = (error['real'], error['predicho'])
    if par not in confusiones:
      confusiones[par] = []
    confusiones[par].append(error)

  print(f"\n📊 CONFUSIONES MÁS COMUNES:")
  for (real, pred), casos in sorted(confusiones.items(), key=lambda x: len(x[1]), reverse=True):
    print(f"  {real} → {pred}: {len(casos)} casos")

  print()

In [ ]:
def mostrarEjemplosErrores(errores, n_ejemplos=4, name=""):
    """Muestra ejemplos visuales de errores de clasificación en 2 columnas"""

    if len(errores) == 0:
        print("¡No hay errores para mostrar!")
        return

    n_mostrar = min(n_ejemplos, len(errores))
    errores_sample = random.sample(errores, k=n_mostrar)

    # Definir 2 columnas
    cols = 2
    rows = math.ceil(n_mostrar / cols)

    fig, axes = plt.subplots(rows, cols, figsize=(10, 5 * rows))
    axes = axes.flatten()  # Convertir a lista para indexar fácilmente

    fig.suptitle('Ejemplos de Errores de Clasificación', fontsize=16, fontweight='bold')

    for i, error in enumerate(errores_sample):
        axes[i].imshow(error['imagen'], cmap='gray', clim=(0,1))
        axes[i].set_title(f'Real: {error["real"].title()}\nPredicho: {error["predicho"].title()}')
        axes[i].axis('off')

    # Ocultar ejes sobrantes si hay menos imágenes que subplots
    for j in range(i+1, len(axes)):
        axes[j].axis('off')

    plt.tight_layout()

    # Guardar en Drive
    output_dir = '/content/drive/MyDrive/TP-Final/'
    os.makedirs(output_dir, exist_ok=True)
    output_path = os.path.join(output_dir, f"{name.replace(' ', '_')}.png")
    plt.savefig(output_path, dpi=300, bbox_inches='tight')


    plt.show()

## PIPELINE DUAL

In [ ]:
import time
import numpy as np
from sklearn.metrics import accuracy_score

def pipeline(
    subsets,
    crearMatrizDescriptores,
    crearMatrizBOW,
    aplicar_tfidf,
    entrenar_svm,
    evaluar_clasificador_svm,
    n_clusters=0,
    K=0,
    batch_size=0,
    clustering_method=None
):

    if (clustering_method not in ["agglomerative_clustering", "minibatchkmeans"]):
      print("Clustering inválido")
      return

    usaKMeans = clustering_method == "minibatchkmeans"

    resultados = []

    n_folds = len(subsets)


    print(f"\n{'='*60}")
    if usaKMeans:
       print(f"Evaluando vocabulario con k={K} y batch_size={batch_size}")
    else:
       print(f"Evaluando vocabulario con {n_clusters} clusters...")
    print(f"{'='*60}\n")

    metrics_bow = []
    metrics_tfidf = []
    tiempos = []

    for fold in range(n_folds):
        print(f"Fold {fold + 1}/{n_folds}")

        # Separo en subsets para cross validation
        val_subset = subsets[fold]
        train_subsets = [s for i, s in enumerate(subsets) if i != fold]
        train_subset = unirSubsets(train_subsets)

        print("Creo matriz de descriptores global")

        # 1. Creao matriz de descriptores global
        descriptores = crearMatrizDescriptores(train_subset['descriptores'])

        # 2. Entreno vocabulario

        inicio = time.time()

        descriptores_entrenamiento = np.vstack(train_subset['descriptores'])

        print("Entreno vocabulario")
        if usaKMeans:
          vocabulario = construirVocabularioMiniBatchKMeans(descriptores_entrenamiento, k=K, batch_size=batch_size)
        else:
          vocabulario = construirVocabularioAgglomerativeClustering(descriptores_entrenamiento, n_clusters)

        fin = time.time()
        tiempo_vocab = fin - inicio
        tiempos.append(tiempo_vocab)

        # 3. Representaciones BoW

        print("Creo matriz BOW")
        X_train_bow = crearMatrizBOW(train_subset['descriptores'], vocabulario, clustering=clustering_method)
        X_val_bow = crearMatrizBOW(val_subset['descriptores'], vocabulario, clustering=clustering_method)

        # # 4. TF-IDF
        X_train_tfidf = aplicar_tfidf(X_train_bow)
        X_val_tfidf = aplicar_tfidf(X_val_bow)

        # 5. Entrenar clasificadores

        print("Entreno clasificador")

        clf_bow = entrenar_svm(X_train_bow, train_subset['etiquetas'])
        clf_tfidf = entrenar_svm(X_train_tfidf, train_subset['etiquetas'])

        # 6. Evaluo los clasificadores

        print("Evaluo")
        resultado_bow = evaluar_clasificador_svm(
            clf_bow, X_val_bow, val_subset['etiquetas'], f"BoW fold {fold+1} (n_cluster={n_clusters})"
        )
        resultado_tfidf = evaluar_clasificador_svm(
            clf_tfidf, X_val_tfidf, val_subset['etiquetas'], f"TF-IDF fold {fold+1} (n_cluster={n_clusters})"
        )

        metrics_bow.append(resultado_bow)
        metrics_tfidf.append(resultado_tfidf)

    # Promedio métricas por fold (accuracy, precision, recall, f1)
    def promediar_metricas(lista_resultados):
        return {
            'accuracy': np.mean([r['accuracy'] for r in lista_resultados]),
            'precision': np.mean([r['precision'] for r in lista_resultados], axis=0),
            'recall': np.mean([r['recall'] for r in lista_resultados], axis=0),
            'f1': np.mean([r['f1'] for r in lista_resultados], axis=0),
            'support': np.sum([r['support'] for r in lista_resultados], axis=0),  # soporte se suma
            'confusion_matrix': np.sum([r['confusion_matrix'] for r in lista_resultados], axis=0)
        }

    promedio_bow = promediar_metricas(metrics_bow)
    promedio_tfidf = promediar_metricas(metrics_tfidf)

    if usaKMeans:
        res = {
        'k' : K,
        'batch_size' : batch_size,
        'n_cluster' : n_clusters,
        'tiempo_promedio_vocab': np.mean(tiempos),
        'bow': promedio_bow,
        'tfidf': promedio_tfidf
        }

    else:
        res = {
        'n_cluster' : n_clusters,
        'tiempo_promedio_vocab': np.mean(tiempos),
        'bow': promedio_bow,
        'tfidf': promedio_tfidf
        }

    return res

In [ ]:
n_clusters = [5, 10, 20, 50, 100, 150, 200]
k_values = [5, 30, 50, 100, 300]
batch_sizes = [200, 500, 800, 1000]

if recalcular_pipeline:
  res_agglomerative = []

  for n_cluster in n_clusters:
    res = pipeline(
      subsets=subsets_daisy,
      crearMatrizDescriptores=crearMatrizDescriptores,
      crearMatrizBOW=crearMatrizBOW,
      aplicar_tfidf=aplicar_tfidf,
      entrenar_svm=entrenar_svm,
      evaluar_clasificador_svm=evaluar_clasificador_svm,
      n_clusters=n_cluster,
      K=None,
      batch_size=None,
      clustering_method="agglomerative_clustering"
      )
    res_agglomerative.append(res)

  res_kmeans = []

  for k in k_values:
    for batch_size in batch_sizes:
      res = pipeline(
        subsets=subsets_sift,
        crearMatrizDescriptores=crearMatrizDescriptores,
        crearMatrizBOW=crearMatrizBOW,
        aplicar_tfidf=aplicar_tfidf,
        entrenar_svm=entrenar_svm,
        evaluar_clasificador_svm=evaluar_clasificador_svm,
        n_clusters=None,
        K=k,
        batch_size=batch_size,
        clustering_method="minibatchkmeans"
        )
      res_kmeans.append(res)

    # Guardamos así no lo calculamos todo el tiempo
    with open("res_agglomerative.pkl", "wb") as f:
        pickle.dump(res_agglomerative, f)
    with open("res_kmeans.pkl", "wb") as f:
        pickle.dump(res_kmeans, f)

else:
    with open("/content/drive/MyDrive/TP-Final/res_agglomerative.pkl", "rb") as f:
        res_agglomerative = pickle.load(f)
    with open("/content/drive/MyDrive/TP-Final/res_kmeans.pkl", "rb") as f:
        res_kmeans = pickle.load(f)

### Resultados PIPELINE

In [ ]:
print(f"{'n_clusters':<6} {'BoW Acc':<10} {'TF-IDF Acc':<12} {'Tiempo Vocabulario (s)':<10} ")
print("-" * 50)
for r in res_agglomerative: # Corrected to iterate through res_agglomerative for n_cluster
    print(f"{r['n_cluster']:<6}  {r['bow']['accuracy']:<12.2f} {r['tfidf']['accuracy']:<10.3f} {r['tiempo_promedio_vocab']:<10.3f}")

print(" ")
print(f"{'k':<6} {'batch_size':<10} {'Acc. BoW':<10} {'Acc. TF-IDF':<10} {'Tiempo(s)':<12} ")
print("-" * 50)
for r in res_kmeans: # Corrected to iterate through res_kmeans for k and batch_size
    print(f"{r['k']:<6} {r['batch_size']:<10} {r['bow']['accuracy']:<10.3f} {r['tfidf']['accuracy']:<10.3f} {r['tiempo_promedio_vocab']:<12.2f}")

In [ ]:
import pandas as pd

def convertir_resultados_a_df(resultados, metodo):
    filas = []
    for r in resultados:
        fila = {
            'metodo': metodo,
            'n_clusters': r.get('n_cluster'),
            'k': r.get('k', None),
            'batch_size': r.get('batch_size', None),
            'tiempo_vocab': r['tiempo_promedio_vocab'],
            'accuracy_bow': r['bow']['accuracy'],
            'accuracy_tfidf': r['tfidf']['accuracy'],
            'f1_macro_bow': r['bow']['f1'].mean(),
            'f1_macro_tfidf': r['tfidf']['f1'].mean()
        }
        filas.append(fila)
    return pd.DataFrame(filas)

df_kmeans = convertir_resultados_a_df(res_kmeans, "MiniBatchKMeans")
df_agglom = convertir_resultados_a_df(res_agglomerative, "Agglomerative")

df_resultados = pd.concat([df_kmeans, df_agglom], ignore_index=True)

In [ ]:
sns.set(style="whitegrid")
plt.figure(figsize=(18, 10))

# 1. Accuracy BoW
plt.subplot(2, 3, 1)
sns.lineplot(data=df_resultados, x='n_clusters', y='accuracy_bow', marker='o')
plt.title('Accuracy (BoW) vs n_clusters')
plt.ylabel('Accuracy')
plt.xlabel('n_clusters')

# 2. Accuracy TF-IDF
plt.subplot(2, 3, 2)
sns.lineplot(data=df_resultados, x='n_clusters', y='accuracy_tfidf', marker='o')
plt.title('Accuracy (TF-IDF) vs n_clusters')
plt.ylabel('Accuracy')
plt.xlabel('n_clusters')

# 3. Tiempo vocabulario
plt.subplot(2, 3, 3)
sns.lineplot(data=df_resultados, x='n_clusters', y='tiempo_vocab', marker='o')
plt.title('Tiempo promedio vocabulario vs n_clusters')
plt.ylabel('Tiempo (seg)')
plt.xlabel('n_clusters')

# 4. F1 macro BoW
plt.subplot(2, 3, 4)
sns.lineplot(data=df_resultados, x='n_clusters', y='f1_macro_bow', marker='o')
plt.title('F1 macro (BoW) vs n_clusters')
plt.ylabel('F1 macro')
plt.xlabel('n_clusters')

# 5. F1 macro TF-IDF
plt.subplot(2, 3, 5)
sns.lineplot(data=df_resultados, x='n_clusters', y='f1_macro_tfidf', marker='o')
plt.title('F1 macro (TF-IDF) vs n_clusters')
plt.ylabel('F1 macro')
plt.xlabel('n_clusters')

# Espacio entre subplots

plt.suptitle("Visualización de resultados para Daisy + Agglomerative Clustering")
plt.tight_layout()
plt.show()

Podemos observar que a partir de los 50 clusters no hay una tendencia significativa de aumento en las métricas. Además, el tiempo se mantiene cercano al rango de los 70-80 segundos para todos los valores. Entre Bag of Words y TF-IDF no vemos una ventaja clara, ya que dan resultados muy similares.

In [ ]:
# Función auxiliar para crear cada heatmap
def plot_heatmap(df, value_col, title, ax, cmap="YlGnBu", fmt=".3f"):
    tabla = df.pivot(index='batch_size', columns='k', values=value_col)
    sns.heatmap(tabla, annot=True, fmt=fmt, cmap=cmap, ax=ax)
    ax.set_title(title)
    ax.set_xlabel("k")
    ax.set_ylabel("batch_size")

# Crear figura: 3 filas, 2 columnas
fig, axes = plt.subplots(3, 2, figsize=(14, 14))

# Heatmaps métricas
plot_heatmap(df_kmeans, 'accuracy_bow', 'Accuracy (BoW)', axes[0, 0])
plot_heatmap(df_kmeans, 'accuracy_tfidf', 'Accuracy (TF-IDF)', axes[0, 1])
plot_heatmap(df_kmeans, 'f1_macro_bow', 'F1 macro (BoW)', axes[1, 0], cmap="magma")
plot_heatmap(df_kmeans, 'f1_macro_tfidf', 'F1 macro (TF-IDF)', axes[1, 1], cmap="magma")

# Heatmap de tiempo
plot_heatmap(df_kmeans, 'tiempo_vocab', 'Tiempo construcción vocabulario (seg)', axes[2, 0], cmap="OrRd", fmt=".2f")
axes[2, 1].axis('off')  # Dejar el último espacio vacío

# Título general
plt.suptitle("Efecto de k y batch_size en SIFT+MiniBatchKMeans", fontsize=18)


En este caso, vemos que con K=30 ya es suficiente para un accuracy elevado, y que con K=300 es casi inmejorable. Hay una leve tendencia a aumentar los parametros con un K más elevado, sin embargo, no hay diferencias significativas al aumentar el batch_size más allá de 200. El tiempo, por su parte, es muy inferior al de Daisy+Agglomerative Clustering, y podemos notar que a medida que aumentamos los clusters, también lo hace el tiempo, aunque nunca superior a los 2 segundos.

In [ ]:
print("🔝 Top 5 resultados según Accuracy (BoW):")
print(df_resultados.sort_values(by='accuracy_bow', ascending=False).head(5))

Si priorizamos el accuracy o el f1, que ya vimos que aumentan de forma muy similar, las mejores combinaciones son con MiniBatchKMeans, y a un costo computacional considerablemente mejor.

## Evaluación final de ambos modelo

In [ ]:
# Seleccionar los mejores resultados según BoW Accuracy
best_agglomerative = max(res_agglomerative, key=lambda r: r['bow']['accuracy'])
best_kmeans = max(res_kmeans, key=lambda r: r['bow']['accuracy'])

# Extraer valores
best_n_cluster = best_agglomerative['n_cluster']
best_k = best_kmeans['k']
best_batch = best_kmeans['batch_size']

print("Mejor Agglomerative:")
print(f"  n_clusters = {best_n_cluster}, Accuracy = {best_agglomerative['bow']['accuracy']:.3f}")

print("Mejor MiniBatchKMeans:")
print(f"  k = {best_k}, batch_size = {best_batch}, Accuracy = {best_kmeans['bow']['accuracy']:.3f}")

In [ ]:
if recalcular_test:
    # Extraer SIFT del set de test
    keypoints_test_sift, descriptores_test_sift, stats_test_sift = extraerSIFTDataSet(imagenes_test, etiquetas_test)
    keypoints_test_daisy, descriptores_test_daisy, stats_test_daisy = extraerDaisyDataSet(imagenes_test, step=45, image_index = 5, radius=30, rings=2, histograms=6, orientations=8)
    # Guardar los resultados de test
    with open('sift_test.pkl', 'wb') as f:
        pickle.dump((keypoints_test_sift, descriptores_test_sift, stats_test_sift), f)

    with open('daisy_test.pkl', 'wb') as f:
        pickle.dump((keypoints_test_daisy, descriptores_test_daisy, stats_test_daisy), f)
else:

    with open('sift_test.pkl', 'rb') as f:
        keypoints_test_sift, descriptores_test_sift, stats_test_sift = pickle.load(f)

    with open('daisy_test.pkl', 'rb') as f:
        keypoints_test_daisy, descriptores_test_daisy, stats_test_daisy = pickle.load(f)

In [ ]:
# Evaluo Daisy + Agglomerative clustering

if recalcular_mejores:

  descs_entrenamiento = np.vstack(descriptores_train_daisy)

  vocab = construirVocabularioAgglomerativeClustering(descs_entrenamiento, n_clusters=best_n_cluster)

  X_train_bow = crearMatrizBOW(descriptores_train_daisy, vocab, clustering='agglomerative_clustering')

  X_test_bow = crearMatrizBOW(descriptores_test_daisy, vocab, clustering='agglomerative_clustering')

  clf_bow = entrenar_svm(X_train_bow, etiquetas_train)

  resultado_daisy = evaluar_clasificador_svm(clf_bow, X_test_bow, etiquetas_test, "Daisy + Agglomerative Clustering")

  if recalcular_mejores:
    with open('res_mejor_daisy.pkl', 'wb') as f:
        pickle.dump(resultado_daisy, f)

else:
  with open('res_mejor_daisy.pkl', 'rb') as f:
    resultado_daisy = pickle.load(f)


print(resultado_daisy["accuracy"])

In [ ]:
# Evaluo SIFT + MiniBatchKMeans

if recalcular_mejores:

  descs_entrenamiento = np.vstack(descriptores_train_sift)

  vocab2 = construirVocabularioMiniBatchKMeans(descs_entrenamiento, k=best_k, batch_size=best_batch)

  X_train_bow = crearMatrizBOW(descriptores_train_sift, vocab2, clustering='minibatchkmeans')

  X_test_bow = crearMatrizBOW(descriptores_test_sift, vocab2, clustering='minibatchkmeans')

  clf_bow = entrenar_svm(X_train_bow, etiquetas_train)

  resultado_sift = evaluar_clasificador_svm(clf_bow, X_test_bow, etiquetas_test, "SIFT + MiniBatchKMeans")

  if recalcular_mejores:
    with open('/content/drive/MyDrive/TP-Final/res_mejor_sift.pkl', 'wb') as f:
        pickle.dump(resultado_sift, f)

else:
  with open('/content/drive/MyDrive/TP-Final/res_mejor_sift.pkl', 'rb') as f:
      resultado_sift = pickle.load(f)

print(resultado_sift["accuracy"])

In [ ]:
resultados = {
    'sift': resultado_sift,
    'daisy': resultado_daisy
}

ks = {
    'sift': best_k,
    'daisy': None
}

metodos = [
    ('sift', 'BoW SIFT Final'),
    ('daisy', 'BoW DAISY Final')
]

In [ ]:
plotConfusionMatrices({'bow': resultado_sift}, best_k, [('bow', 'BoW Final')])
plotConfusionMatrices({'bow': resultado_daisy}, "-", [('bow', 'BoW Final')])

In [ ]:
testing_set = {
    'imagenes': imagenes_test,
    'etiquetas': etiquetas_test,
    'nombres': nombres_test
}
errores_test_daisy = analizar_errores({'bow': resultado_daisy}, 'bow', testing_set)
imprimirErrores(errores_test_daisy, k=None, metodo='bow', total=len(etiquetas_test), name="DAISY")

errores_test_sift = analizar_errores({'bow': resultado_sift}, 'bow', testing_set)
imprimirErrores(errores_test_sift, k=best_k, metodo='bow', total=len(etiquetas_test), name="SIFT")

In [ ]:
mostrarEjemplosErrores(errores_test_daisy, 2, name="DAISY + Agglomerative Clustering")
mostrarEjemplosErrores(errores_test_sift, 4, name="SIFT + MiniBatchKMeans")

In [ ]:
def graficar_sift_vs_daisy(imagen, kp_sift, kp_daisy, step=45, radius=30, rings=2, histograms=6, orientations=8):
    """
    Muestra comparación entre SIFT, DAISY simple y DAISY detallado (visualize=True).

    Params:
        imagen: imagen en escala de grises o RGB
        kp_sift: lista de puntos (KeyPoint o coordenadas)
        kp_daisy: lista de puntos (KeyPoint o coordenadas)
    """
    # Convertir a escala de grises para DAISY
    img_gray = color.rgb2gray(imagen) if imagen.ndim == 3 else imagen

    # Normalizar para OpenCV
    img_cv = (img_gray * 255).astype('uint8') if img_gray.dtype != 'uint8' else img_gray

    # Convertir keypoints DAISY a KeyPoint si es necesario
    def convertir(lista, size=10):
        return [cv2.KeyPoint(float(x), float(y), size) for (x, y) in lista]

    if not isinstance(kp_sift[0], cv2.KeyPoint):
        kp_sift = convertir(kp_sift)
    if not isinstance(kp_daisy[0], cv2.KeyPoint):
        kp_daisy = convertir(kp_daisy)

    # Dibujar keypoints
    img_sift = cv2.drawKeypoints(img_cv, kp_sift, None, flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)

    # DAISY detallado (visualize=True)
    _, daisy_vis = daisy(img_gray, step=step, radius=radius, rings=rings,
                         histograms=histograms, orientations=orientations,
                         visualize=True)

    # Mostrar
    fig, axs = plt.subplots(1, 2, figsize=(15, 5))
    axs[0].imshow(cv2.cvtColor(img_sift, cv2.COLOR_BGR2RGB))
    axs[0].set_title(f"SIFT Keypoints ({len(kp_sift)})", fontsize=16, fontweight='bold')
    axs[0].axis('off')


    axs[1].imshow(daisy_vis, cmap='gray')
    axs[1].set_title("DAISY Structure", fontsize=16, fontweight='bold')
    axs[1].axis('off')

    plt.tight_layout()
    plt.show()

In [ ]:
imagen = imagenes_test[7]
kp_sift = keypoints_test_sift[7]
kp_daisy = keypoints_test_daisy[7]

graficar_sift_vs_daisy(imagen, kp_sift, kp_daisy)